In [18]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

In [19]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [20]:
tf.keras.backend.clear_session()

In [21]:
subject = 'Sergipe - Consumo de Cimento (t)'
start_index = 0
split_index = 191 #Referente aos 230 anos de input 
window_size = 36
train_split = split_index + 1 - 30

In [22]:
data = pd.read_csv('2003_mo_model_input_SE.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
data

,Unnamed: 0,Sergipe - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Sergipe - value,Sergipe - Produção de Cimento (t),Sergipe - PIB - Estadual,Sergipe - PIB - Construção Civil,Sergipe - PIB - Per Capita,Sergipe - PIB - Preços de Mercado,Sergipe - Desemprego,Sergipe - Consumo de Cimento (t)
0,2003-1,0.669217,0.724032,11.520143,1.639718,1.036534,1.772069e+08,7.330309e+06,0.969649,0.339447,152.791400,2.159732e+07,1.260978e+06,9.960678,2.029642e+07,8.389943,23.616
1,2003-2,0.669542,0.690297,11.189862,1.378899,0.993449,1.773884e+08,7.335910e+06,0.950783,0.341257,151.964054,2.162153e+07,1.262134e+06,9.961956,2.030238e+07,8.384067,21.357
2,2003-3,0.669868,0.669681,10.820792,1.924317,0.973020,1.775699e+08,7.341511e+06,0.938332,0.343016,154.467758,2.164574e+07,1.263289e+06,9.963234,2.030833e+07,8.378190,20.059
3,2003-4,0.670193,0.660494,10.417840,1.331174,0.940489,1.777514e+08,7.347112e+06,0.926401,0.344722,155.913400,2.166995e+07,1.264444e+06,9.964512,2.031429e+07,8.372313,19.920
4,2003-5,0.670519,0.648337,9.959690,1.736072,0.917493,1.779329e+08,7.352713e+06,0.951683,0.346174,157.467159,2.169416e+07,1.265599e+06,9.965790,2.032025e+07,8.366437,19.962
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,2022-8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.557749,169.752591,NaN,NaN,NaN,NaN,NaN,34.037
236,2022-9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.556659,169.621865,NaN,NaN,NaN,NaN,NaN,32.537
237,2022-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.555237,169.855545,NaN,NaN,NaN,NaN,NaN,35.215
238,2022-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.553445,170.188727,NaN,NaN,NaN,NaN,NaN,34.707


In [23]:
input_data = data.iloc[:split_index + 1,1:-1]
mean = np.mean(input_data, axis=0)
stddev =  np.std(input_data, axis=0)
input_data = ((input_data - mean) /stddev)
input_data

,Sergipe - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Sergipe - value,Sergipe - Produção de Cimento (t),Sergipe - PIB - Estadual,Sergipe - PIB - Construção Civil,Sergipe - PIB - Per Capita,Sergipe - PIB - Preços de Mercado,Sergipe - Desemprego
0,-2.191597,2.723741,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582,-1.329544,-1.638248,-1.769739,-0.221258,0.566843,-2.721827,-0.858763
1,-2.153964,2.350880,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356,-1.294452,-1.660220,-1.748579,-0.190897,0.585366,-2.626858,-0.861105
2,-2.116331,2.123016,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895,-1.260345,-1.593729,-1.727419,-0.160535,0.603889,-2.531890,-0.863447
3,-2.078698,2.021477,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628,-1.227259,-1.555337,-1.706259,-0.130173,0.622411,-2.436921,-0.865789
4,-2.041065,1.887113,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624,-1.199102,-1.514074,-1.685099,-0.099812,0.640934,-2.341953,-0.868132
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,1.375707,-2.010387,-0.572934,-1.281958,0.589021,1.365911,0.389193,-1.749976,0.926582,-1.147300,1.088440,-1.830495,-1.273118,-0.375044,1.205390
188,1.367460,-1.870713,-0.588777,-1.358588,1.043728,1.376610,0.370392,-1.593005,0.922177,-1.137939,1.080037,-1.823605,-1.263526,-0.370593,1.201393
189,1.359213,-1.806230,-0.612606,-1.511565,1.387010,1.387308,0.351592,-1.351489,0.923529,-1.097718,1.071634,-1.816716,-1.253934,-0.366142,1.197396
190,1.350966,-1.727496,-0.640956,-1.421708,1.815728,1.398006,0.332791,-1.198492,0.936477,-1.086004,1.063231,-1.809827,-1.244342,-0.361691,1.193399


In [24]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-12)
target_data

0      19.642
1      17.778
2      21.180
3      18.824
4      16.535
        ...  
235       NaN
236       NaN
237       NaN
238       NaN
239       NaN
Name: Sergipe - Consumo de Cimento (t), Length: 240, dtype: float64

In [25]:
# input para treinamento
train_input = input_data.iloc[start_index:train_split]
train_input

,Sergipe - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Sergipe - value,Sergipe - Produção de Cimento (t),Sergipe - PIB - Estadual,Sergipe - PIB - Construção Civil,Sergipe - PIB - Per Capita,Sergipe - PIB - Preços de Mercado,Sergipe - Desemprego
0,-2.191597,2.723741,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582,-1.329544,-1.638248,-1.769739,-0.221258,0.566843,-2.721827,-0.858763
1,-2.153964,2.350880,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356,-1.294452,-1.660220,-1.748579,-0.190897,0.585366,-2.626858,-0.861105
2,-2.116331,2.123016,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895,-1.260345,-1.593729,-1.727419,-0.160535,0.603889,-2.531890,-0.863447
3,-2.078698,2.021477,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628,-1.227259,-1.555337,-1.706259,-0.130173,0.622411,-2.436921,-0.865789
4,-2.041065,1.887113,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624,-1.199102,-1.514074,-1.685099,-0.099812,0.640934,-2.341953,-0.868132
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,1.387754,-0.214006,-0.607704,0.059280,-1.233012,1.031384,0.819304,-0.883659,1.521513,-0.132792,1.143453,-1.270245,-1.549275,-0.543399,1.342411
158,1.390522,-0.434717,-0.620523,0.222389,-1.299304,1.042716,0.808136,-0.950771,1.498482,-0.189562,1.147046,-1.306762,-1.539477,-0.525912,1.335629
159,1.393291,-0.524091,-0.631530,0.191929,-1.248662,1.054049,0.796969,-1.028465,1.475235,-0.253067,1.150639,-1.343278,-1.529680,-0.508425,1.328848
160,1.396059,-0.614500,-0.640320,0.385687,-1.068274,1.065381,0.785801,-1.103668,1.452580,-0.330172,1.154232,-1.379795,-1.519882,-0.490938,1.322066


In [26]:
# Alvo para treinamento
train_target = target_data.iloc[start_index:train_split]
train_target

0      19.642
1      17.778
2      21.180
3      18.824
4      16.535
        ...  
157    37.579
158    48.016
159    38.263
160    40.413
161    34.053
Name: Sergipe - Consumo de Cimento (t), Length: 162, dtype: float64

In [27]:
def create_batches(t_input, t_target, window_size, start_from):
    
    X_batches = []
    y_batches = []

    train_input_values = t_input.values 

    for i in range(len(t_input) - window_size):
        
        X_window = train_input_values[i:i+window_size, :]
        y_target = t_target[start_from+i+window_size]

        X_batches.append(X_window)
        y_batches.append(y_target)

    return np.array(X_batches), np.array(y_batches)

In [28]:
reshaped_train, reshaped_target = create_batches(train_input, 
                                                 train_target, 
                                                 window_size, 
                                                 start_index)
reshaped_train.shape

(126, 36, 15)

In [29]:
# input para treinamento
test_input = input_data.iloc[train_split - window_size:split_index + 1]
test_input

,Sergipe - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Sergipe - value,Sergipe - Produção de Cimento (t),Sergipe - PIB - Estadual,Sergipe - PIB - Construção Civil,Sergipe - PIB - Per Capita,Sergipe - PIB - Preços de Mercado,Sergipe - Desemprego
126,0.374606,0.888984,-0.460555,-1.131704,-0.368821,0.651397,0.944085,0.045243,0.708889,1.481103,0.853614,-0.111030,-0.618059,1.032015,0.905037
127,0.381094,0.954254,-0.440372,-0.613483,-0.328087,0.664707,0.947319,0.061828,0.753914,1.474131,0.869229,-0.156897,-0.659078,0.974979,0.923945
128,0.387582,1.045217,-0.419247,-0.542919,-0.176031,0.678017,0.950553,0.046225,0.799633,1.440003,0.884845,-0.202764,-0.700096,0.917944,0.942853
129,0.394069,1.176395,-0.397019,-1.334517,-0.113037,0.691327,0.953786,0.032522,0.849863,1.408120,0.900461,-0.248632,-0.741114,0.860908,0.961762
130,0.400557,1.303259,-0.376532,-1.511762,-0.022703,0.704637,0.957020,0.042757,0.896706,1.375556,0.916077,-0.294499,-0.782133,0.803873,0.980670
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,1.375707,-2.010387,-0.572934,-1.281958,0.589021,1.365911,0.389193,-1.749976,0.926582,-1.147300,1.088440,-1.830495,-1.273118,-0.375044,1.205390
188,1.367460,-1.870713,-0.588777,-1.358588,1.043728,1.376610,0.370392,-1.593005,0.922177,-1.137939,1.080037,-1.823605,-1.263526,-0.370593,1.201393
189,1.359213,-1.806230,-0.612606,-1.511565,1.387010,1.387308,0.351592,-1.351489,0.923529,-1.097718,1.071634,-1.816716,-1.253934,-0.366142,1.197396
190,1.350966,-1.727496,-0.640956,-1.421708,1.815728,1.398006,0.332791,-1.198492,0.936477,-1.086004,1.063231,-1.809827,-1.244342,-0.361691,1.193399


In [30]:
reshaped_test, reshaped_test_target = create_batches(test_input, 
                                                     target_data, 
                                                     window_size, 
                                                     train_split - window_size)
reshaped_test.shape 

(30, 36, 15)

In [31]:
def validation_splitter(arr, div_factor, add_factor=0):
    split_factor = len(arr) // div_factor
    positions_to_drop_index = []
    positions_to_drop = []
    for i in range(split_factor):
        pos = len(arr) - (i * div_factor + 1)
        positions_to_drop_index.append(pos)
        positions_to_drop.append(pos + add_factor)
    
    arr_droped = arr[positions_to_drop]
    arr_result = np.delete(arr, positions_to_drop_index, axis=0)
    
    return arr_result, arr_droped

In [32]:
# Rede Neural Recorrente com optmizador Estocástico
def lstm_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    train, train_val = validation_splitter(train_input, 7)
    target,target_val = validation_splitter(train_target, 7)
#     display(train.shape)
#     display(train_val.shape)
#     display(target.shape)
#     display(target_val.shape)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.LSTM(36, activation='tanh', 
                             return_sequences=True, 
                             input_shape=(reshaped_train.shape[1],
                                          reshaped_train.shape[2])),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.LSTM(180,activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train, 
                        target, 
                        epochs=10000,
                        validation_data=(train_val,
                                         target_val),
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [33]:
def get_a_good_seed(train_input, train_target, test_input, test_target):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(50)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = lstm_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [34]:
winner_seed = get_a_good_seed(reshaped_train, reshaped_target, reshaped_test, reshaped_test_target)
print(f"\n\nfinal_seed: {winner_seed}")

[2944620158, 3344566210, 2896466950, 2966182252, 2093883138, 1715788591, 2004662025, 2981110686, 3251962815, 957862835, 3429410530, 793583122, 1375168535, 2969261801, 361314469, 1991718443, 1118167409, 217668182, 3554061719, 2132519452, 3486465198, 494472616, 2762284267, 1455504333, 251871995, 667073812, 1795959782, 2445017505, 2573448135, 3028896358, 195914888, 3834307399, 1969983493, 1109209367, 3508485590, 3287023368, 551679259, 1300206353, 2261455521, 3365639941, 2567217087, 1266823089, 41599635, 2533337923, 2719322647, 356421713, 796752358, 3006656688, 1151345128, 157434147]


Step: 0 ___________________________________________
val_loss: 8.295879364013672
winner_seed: 2944620158


Step: 1 ___________________________________________
val_loss: 10.308768272399902


Step: 2 ___________________________________________
val_loss: 6.8928937911987305
winner_seed: 2896466950


Step: 3 ___________________________________________
val_loss: 9.337800025939941


Step: 4 _________________________

In [35]:
trained_model, history = lstm_model(reshaped_train, 
                                    reshaped_target, 
                                    want_verbose=1, 
                                    seed=winner_seed)

Epoch 1/10000
4/4 [==============================] - 2s 114ms/step - loss: 1637.7188 - val_loss: 104.1662
Epoch 2/10000
4/4 [==============================] - 0s 7ms/step - loss: 89.3046 - val_loss: 116.9563
Epoch 3/10000
4/4 [==============================] - 0s 7ms/step - loss: 81.4625 - val_loss: 67.1756
Epoch 4/10000
4/4 [==============================] - 0s 7ms/step - loss: 64.4514 - val_loss: 146.9881
Epoch 5/10000
4/4 [==============================] - 0s 7ms/step - loss: 142.3681 - val_loss: 109.4797
Epoch 6/10000
4/4 [==============================] - 0s 7ms/step - loss: 72.6437 - val_loss: 70.1726
Epoch 7/10000
4/4 [==============================] - 0s 8ms/step - loss: 28.4184 - val_loss: 31.0412
Epoch 8/10000
4/4 [==============================] - 0s 7ms/step - loss: 19.7103 - val_loss: 61.2254
Epoch 9/10000
4/4 [==============================] - 0s 7ms/step - loss: 59.9727 - val_loss: 99.1729
Epoch 10/10000
4/4 [==============================] - 0s 8ms/step - loss: 49.5819 

4/4 [==============================] - 0s 7ms/step - loss: 8.7213 - val_loss: 16.6838
Epoch 162/10000
4/4 [==============================] - 0s 8ms/step - loss: 12.1120 - val_loss: 14.7919
Epoch 163/10000
4/4 [==============================] - 0s 8ms/step - loss: 8.7522 - val_loss: 16.0539
Epoch 164/10000
4/4 [==============================] - 0s 7ms/step - loss: 7.8228 - val_loss: 15.6034
Epoch 165/10000
4/4 [==============================] - 0s 7ms/step - loss: 9.0338 - val_loss: 14.3855
Epoch 166/10000
4/4 [==============================] - 0s 7ms/step - loss: 8.9697 - val_loss: 17.8422
Epoch 167/10000
4/4 [==============================] - 0s 7ms/step - loss: 8.8073 - val_loss: 16.7849
Epoch 168/10000
4/4 [==============================] - 0s 7ms/step - loss: 8.1214 - val_loss: 18.1039
Epoch 169/10000
4/4 [==============================] - 0s 7ms/step - loss: 7.4041 - val_loss: 16.4889
Epoch 170/10000
4/4 [==============================] - 0s 7ms/step - loss: 8.7351 - val_loss: 17.

4/4 [==============================] - 0s 7ms/step - loss: 7.9584 - val_loss: 19.3948
Epoch 322/10000
4/4 [==============================] - 0s 7ms/step - loss: 9.1716 - val_loss: 13.1637
Epoch 323/10000
4/4 [==============================] - 0s 7ms/step - loss: 7.5116 - val_loss: 14.0322
Epoch 324/10000
4/4 [==============================] - 0s 7ms/step - loss: 6.9069 - val_loss: 13.5305
Epoch 325/10000
4/4 [==============================] - 0s 7ms/step - loss: 7.2288 - val_loss: 13.4961
Epoch 326/10000
4/4 [==============================] - 0s 7ms/step - loss: 6.3082 - val_loss: 12.8027
Epoch 327/10000
4/4 [==============================] - 0s 7ms/step - loss: 6.4334 - val_loss: 12.5303
Epoch 328/10000
4/4 [==============================] - 0s 7ms/step - loss: 6.2553 - val_loss: 12.3843
Epoch 329/10000
4/4 [==============================] - 0s 7ms/step - loss: 7.4532 - val_loss: 13.6870
Epoch 330/10000
4/4 [==============================] - 0s 7ms/step - loss: 7.0415 - val_loss: 13.6

4/4 [==============================] - 0s 7ms/step - loss: 5.1943 - val_loss: 12.0842
Epoch 482/10000
4/4 [==============================] - 0s 7ms/step - loss: 6.4152 - val_loss: 14.8841
Epoch 483/10000
4/4 [==============================] - 0s 7ms/step - loss: 6.4548 - val_loss: 13.5711
Epoch 484/10000
4/4 [==============================] - 0s 7ms/step - loss: 5.2399 - val_loss: 11.4150
Epoch 485/10000
4/4 [==============================] - 0s 7ms/step - loss: 5.8494 - val_loss: 11.5189
Epoch 486/10000
4/4 [==============================] - 0s 7ms/step - loss: 5.3694 - val_loss: 10.8650
Epoch 487/10000
4/4 [==============================] - 0s 7ms/step - loss: 6.2035 - val_loss: 14.7403
Epoch 488/10000
4/4 [==============================] - 0s 7ms/step - loss: 7.0723 - val_loss: 13.9335
Epoch 489/10000
4/4 [==============================] - 0s 7ms/step - loss: 6.6229 - val_loss: 12.3327
Epoch 490/10000
4/4 [==============================] - 0s 7ms/step - loss: 6.4306 - val_loss: 15.5

4/4 [==============================] - 0s 7ms/step - loss: 6.1068 - val_loss: 11.2990
Epoch 642/10000
4/4 [==============================] - 0s 7ms/step - loss: 5.2527 - val_loss: 11.5311
Epoch 643/10000
4/4 [==============================] - 0s 7ms/step - loss: 5.1816 - val_loss: 13.7181
Epoch 644/10000
4/4 [==============================] - 0s 7ms/step - loss: 5.4176 - val_loss: 11.7837
Epoch 645/10000
4/4 [==============================] - 0s 7ms/step - loss: 4.9244 - val_loss: 9.4612
Epoch 646/10000
4/4 [==============================] - 0s 7ms/step - loss: 5.2710 - val_loss: 12.0978
Epoch 647/10000
4/4 [==============================] - 0s 7ms/step - loss: 5.8559 - val_loss: 11.3857
Epoch 648/10000
4/4 [==============================] - 0s 7ms/step - loss: 5.0824 - val_loss: 11.4734
Epoch 649/10000
4/4 [==============================] - 0s 7ms/step - loss: 4.7854 - val_loss: 10.9761
Epoch 650/10000
4/4 [==============================] - 0s 8ms/step - loss: 5.0414 - val_loss: 8.848

4/4 [==============================] - 0s 7ms/step - loss: 4.2004 - val_loss: 12.6696
Epoch 802/10000
4/4 [==============================] - 0s 7ms/step - loss: 4.6315 - val_loss: 11.3888
Epoch 803/10000
4/4 [==============================] - 0s 7ms/step - loss: 4.6150 - val_loss: 11.7325
Epoch 804/10000
4/4 [==============================] - 0s 7ms/step - loss: 4.5482 - val_loss: 14.2906
Epoch 805/10000
4/4 [==============================] - 0s 7ms/step - loss: 4.7787 - val_loss: 14.3865
Epoch 806/10000
4/4 [==============================] - 0s 7ms/step - loss: 4.9354 - val_loss: 10.0644
Epoch 807/10000
4/4 [==============================] - 0s 7ms/step - loss: 3.9149 - val_loss: 8.7772
Epoch 808/10000
4/4 [==============================] - 0s 7ms/step - loss: 3.9940 - val_loss: 9.7474
Epoch 809/10000
4/4 [==============================] - 0s 7ms/step - loss: 4.1781 - val_loss: 11.7526
Epoch 810/10000
4/4 [==============================] - 0s 7ms/step - loss: 5.2044 - val_loss: 9.6193

Epoch 962/10000
4/4 [==============================] - 0s 7ms/step - loss: 4.7587 - val_loss: 8.6253
Epoch 963/10000
4/4 [==============================] - 0s 7ms/step - loss: 4.3482 - val_loss: 14.2232
Epoch 964/10000
4/4 [==============================] - 0s 7ms/step - loss: 4.6200 - val_loss: 10.1612
Epoch 965/10000
4/4 [==============================] - 0s 7ms/step - loss: 4.4947 - val_loss: 9.5182
Epoch 966/10000
4/4 [==============================] - 0s 7ms/step - loss: 4.0007 - val_loss: 8.8568
Epoch 967/10000
4/4 [==============================] - 0s 7ms/step - loss: 3.8423 - val_loss: 9.9707
Epoch 968/10000
4/4 [==============================] - 0s 7ms/step - loss: 4.2579 - val_loss: 9.2231
Epoch 969/10000
4/4 [==============================] - 0s 7ms/step - loss: 4.1323 - val_loss: 9.7928
Epoch 970/10000
4/4 [==============================] - 0s 7ms/step - loss: 3.7961 - val_loss: 8.9932
Epoch 971/10000
4/4 [==============================] - 0s 7ms/step - loss: 3.8205 - val_l

4/4 [==============================] - 0s 7ms/step - loss: 3.9044 - val_loss: 10.7513
Epoch 1123/10000
4/4 [==============================] - 0s 7ms/step - loss: 3.5165 - val_loss: 10.6224
Epoch 1124/10000
4/4 [==============================] - 0s 7ms/step - loss: 3.8202 - val_loss: 9.8395
Epoch 1125/10000
4/4 [==============================] - 0s 7ms/step - loss: 3.6012 - val_loss: 7.8749
Epoch 1126/10000
4/4 [==============================] - 0s 7ms/step - loss: 4.1429 - val_loss: 8.2714
Epoch 1127/10000
4/4 [==============================] - 0s 7ms/step - loss: 4.2088 - val_loss: 8.6753
Epoch 1128/10000
4/4 [==============================] - 0s 7ms/step - loss: 3.9543 - val_loss: 8.7465
Epoch 1129/10000
4/4 [==============================] - 0s 7ms/step - loss: 3.8797 - val_loss: 13.6609
Epoch 1130/10000
4/4 [==============================] - 0s 7ms/step - loss: 4.0613 - val_loss: 9.1273
Epoch 1131/10000
4/4 [==============================] - 0s 7ms/step - loss: 3.7416 - val_loss: 9

4/4 [==============================] - 0s 7ms/step - loss: 3.9034 - val_loss: 9.5465
Epoch 1283/10000
4/4 [==============================] - 0s 7ms/step - loss: 3.5059 - val_loss: 8.1276
Epoch 1284/10000
4/4 [==============================] - 0s 7ms/step - loss: 3.7307 - val_loss: 10.3531
Epoch 1285/10000
4/4 [==============================] - 0s 7ms/step - loss: 3.3967 - val_loss: 8.8327
Epoch 1286/10000
4/4 [==============================] - 0s 7ms/step - loss: 4.3784 - val_loss: 8.4566
Epoch 1287/10000
4/4 [==============================] - 0s 7ms/step - loss: 3.4855 - val_loss: 9.1178
Epoch 1288/10000
4/4 [==============================] - 0s 7ms/step - loss: 3.5581 - val_loss: 8.1194
Epoch 1289/10000
4/4 [==============================] - 0s 7ms/step - loss: 3.0034 - val_loss: 10.7103
Epoch 1290/10000
4/4 [==============================] - 0s 7ms/step - loss: 3.5439 - val_loss: 11.4476
Epoch 1291/10000
4/4 [==============================] - 0s 7ms/step - loss: 3.8349 - val_loss: 8

4/4 [==============================] - 0s 8ms/step - loss: 3.7208 - val_loss: 9.3638
Epoch 1443/10000
4/4 [==============================] - 0s 7ms/step - loss: 3.0009 - val_loss: 8.1377
Epoch 1444/10000
4/4 [==============================] - 0s 8ms/step - loss: 3.3402 - val_loss: 8.4804
Epoch 1445/10000
4/4 [==============================] - 0s 8ms/step - loss: 3.6636 - val_loss: 7.6366
Epoch 1446/10000
4/4 [==============================] - 0s 7ms/step - loss: 3.1533 - val_loss: 12.3134
Epoch 1447/10000
4/4 [==============================] - 0s 7ms/step - loss: 3.4611 - val_loss: 8.6930
Epoch 1448/10000
4/4 [==============================] - 0s 8ms/step - loss: 3.7939 - val_loss: 7.5638
Epoch 1449/10000
4/4 [==============================] - 0s 7ms/step - loss: 3.7789 - val_loss: 13.9242
Epoch 1450/10000
4/4 [==============================] - 0s 8ms/step - loss: 3.8417 - val_loss: 7.8278
Epoch 1451/10000
4/4 [==============================] - 0s 8ms/step - loss: 3.2793 - val_loss: 7.

In [36]:
def mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    results_data = []

    for i in range(len(test_target)):
        prediction = model.predict(test_input[i:i+1])
        target = test_target[start_target + i]
        error = np.abs(prediction - target)
        errors.append(error)
        error_percent.append(error/target)
        results_data.append([f"Month-{i + 1}", 
                             prediction[0][0], 
                             target, 
                             error[0][0]])

    df_results = pd.DataFrame(results_data, columns=["Month", "Prediction", "Target", "Error"])

    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return df_results, mae, mape

In [37]:
df_results, mae, mape = mae_mape_calculator(trained_model, 
                                            reshaped_test, 
                                            reshaped_test_target, 
                                            start_index)
pd.set_option('display.max_columns', None)
df_results.T

1/1 [==============================] - 0s 14ms/step


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
Month,Month-1,Month-2,Month-3,Month-4,Month-5,Month-6,Month-7,Month-8,Month-9,Month-10,Month-11,Month-12,Month-13,Month-14,Month-15,Month-16,Month-17,Month-18,Month-19,Month-20,Month-21,Month-22,Month-23,Month-24,Month-25,Month-26,Month-27,Month-28,Month-29,Month-30
Prediction,35.997047,34.984089,34.445477,34.161217,33.97538,33.824486,33.702152,33.651733,33.607346,33.553478,33.508385,33.504082,33.387379,33.344166,33.356876,33.311115,33.201309,33.046021,33.021568,33.073936,33.110146,33.146404,33.169132,33.181847,33.199295,33.200497,33.186398,33.172787,33.185246,33.212814
Target,36.174,38.394,35.688,42.076,39.875,38.984,41.652,33.904,29.447,29.197,26.297,29.209,28.158,31.132,30.195,31.537,30.748,30.734,36.567,30.849,31.021,31.437,31.533,26.431,25.978,29.86,39.045,45.213,42.237,39.732
Error,0.176952,3.409912,1.242523,7.914783,5.89962,5.159515,7.949848,0.252266,4.160345,4.356478,7.211384,4.295082,5.229378,2.212166,3.161877,1.774115,2.45331,2.312021,3.545433,2.224936,2.089146,1.709404,1.636131,6.750847,7.221294,3.340496,5.858601,12.040215,9.051754,6.519184


In [38]:
display(mae)
display(mape)

4.371967

0.12874822

In [39]:
def year_mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    
    target_sum = 0
    prediction_sum = 0
    
    for i in range(len(test_target)):
        if i % 12 == 0 and i != 0:
            error = np.abs(target_sum - prediction_sum)
            errors.append(error)
            error_percent.append(error / target_sum)
            print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
            target_sum = 0
            prediction_sum = 0
            
        prediction = model.predict(test_input[i:i+1])
        target_sum += test_target[start_target + i]
        prediction_sum += prediction
        
    error = np.abs(target_sum - prediction_sum)
    errors.append(error)
    error_percent.append(error / target_sum)
    print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
        
    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return errors, mae, mape

In [40]:
errors, mae, mape = year_mae_mape_calculator(trained_model,
                                             reshaped_test, 
                                             reshaped_test_target,
                                             start_index)
display(errors)
display(mae)
display(mape)

1/1 [==============================] - 0s 14ms/step
Ano-0: |Prediction[[408.9149]] - Target[420.89700000000005]| =  Error: [[11.982117]]; MAPE:[[0.02846805]]
1/1 [==============================] - 0s 14ms/step
Ano-0: |Prediction[[398.34988]] - Target[370.34200000000004]| =  Error: [[28.007874]]; MAPE:[[0.07562705]]
1/1 [==============================] - 0s 14ms/step
Ano-5: |Prediction[[199.15704]] - Target[222.065]| =  Error: [[22.907959]]; MAPE:[[0.1031588]]


[array([[11.982117]], dtype=float32),
 array([[28.007874]], dtype=float32),
 array([[22.907959]], dtype=float32)]

20.965982

0.06908464